# Library Usage Playground

This Jupyter notebook is simply here to experiment and find any limitations with using AlphaVantage, BackTrader and other tools. 

## AlphaVantage Usage 
AlphaVantage is a free API for getting historical stock data as well as live prices. This could be useful for backtesting but depending on the communication through brokers, likely won't be used for live trading. 

In [4]:
# Imports 
from alpha_vantage.timeseries import TimeSeries
import json  
import numpy as np 
import pandas as pd
import os 
import matplotlib.pyplot as plt
import backtrader as bt 

In [5]:
# Load the API Key once using the 'secrets' library 
with open('../secrets/secrets.json') as f:
    vals = json.load(f)
    API_KEY = vals['ALPHA_VANTAGE_API_KEY']

In [6]:
# Input start and end dates
symbol = 'TSLA'
start_date = '2003-01-01'
end_date = '2023-01-01'
ts = TimeSeries(key=API_KEY, output_format='pandas')
# Fetch daily data
data, meta_data = ts.get_daily(
    symbol=symbol,
    outputsize='full'
)

# Convert index to datetime and sort the data
data.index = pd.to_datetime(data.index)
data = data.sort_index(ascending=True)

# Filter data by the provided date range
data = data.loc[start_date:end_date]
data.head()

# Save the final dataframe to CSV
if not os.path.exists('data'):
    os.makedirs('data')
csv_filename = f"data/AAPL_Testing.csv"
data.to_csv(csv_filename)


In [7]:
# Compute some basic technical indicators 
# Exponential Moving Average (EMA) - 200 Periods
data['EMA_200'] = data['4. close'].ewm(span=200, adjust=False).mean()

# Moving Average Convergence Divergence (MACD)
data['EMA_12'] = data['4. close'].ewm(span=12, adjust=False).mean()
data['EMA_26'] = data['4. close'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['EMA_12'] - data['EMA_26']
data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()

data.dropna(inplace=True)
# Save to CSV 
if not os.path.exists('data'):
    os.makedirs('data')
data.to_csv('data/AAPL_Testing.csv')
data.head()

,1. open,2. high,3. low,4. close,5. volume,EMA_200,EMA_12,EMA_26,MACD,Signal_Line
date,,,,,,,,,,
2010-06-29,19.00,25.0000,17.54,23.89,18766300.0,23.890000,23.890000,23.890000,0.000000,0.000000
2010-06-30,25.79,30.4192,23.30,23.83,17187100.0,23.889403,23.880769,23.885556,-0.004786,-0.000957
2010-07-01,25.00,25.9200,20.27,21.96,8218800.0,23.870205,23.585266,23.742922,-0.157656,-0.032297
2010-07-02,23.00,23.1000,18.71,19.20,5139800.0,23.823735,22.910610,23.406409,-0.495799,-0.124997
2010-07-06,20.00,20.0000,15.83,16.11,6866900.0,23.746982,21.864362,22.865934,-1.001572,-0.300312


## Backtrader Usage 
Backtrader is the most popular backtrading framework for testing strategies in Python. We will now attempt to backtest the extremely basic trading strategy where if the EMA(200) is below the current price and the MACD has crossed the Signal Line while both are below 0. 

In [12]:
# 1. Define Custom Data Feed
class MyCSVData(bt.feeds.GenericCSVData):
    lines = ('ema_200', 'ema_12', 'ema_26', 'macd', 'signal_line',)
    params = (
        ('dtformat', '%Y-%m-%d'),  # Only date, no time
        ('datetime', 0),
        ('open', 1),
        ('high', 2),
        ('low', 3),
        ('close', 4),
        ('volume', 5),
        ('openinterest', -1),
        ('ema_200', 6),
        ('ema_12', 7),
        ('ema_26', 8),
        ('macd', 9),
        ('signal_line', 10),
    )

# 2. Define Strategy
class SimpleMACDStrategy(bt.Strategy):
    def __init__(self):
        self.macd = self.datas[0].macd
        self.signal_line = self.datas[0].signal_line
        self.close_price = self.datas[0].close
        self.ema_200 = self.datas[0].ema_200
        self.order = None  # Track pending orders

    def next(self):
        # Check if we have an open order
        if self.order:
            return  # Wait for pending order to complete

        if not self.position:
            if self.macd[0] > self.signal_line[0] and self.ema_200[0] < self.close_price[0] and self.macd[0] < 0 and self.signal_line[0] < 0:  
                cash = self.broker.get_cash()
                size = int(cash / self.close_price[0])
                if size > 0:
                    print(f'Available cash: {cash}, Calculated size: {size}')
                    print('Attempting to BUY')
                    self.order = self.buy(size=size)
        else:
            if self.macd[0] < self.signal_line[0] or self.ema_200[0] > self.close_price[0] or self.macd[0] > 0 or self.signal_line[0] > 0:
                print('Attempting to SELL')
                self.order = self.sell(size=self.position.size)

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                print(f'BUY executed, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}')
            elif order.issell():
                print(f'SELL executed, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}')
            self.order = None  # Reset order
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print('Order Canceled/Margin/Rejected')
            self.order = None

# 3. Run Backtest
def run_backtest():
    cerebro = bt.Cerebro()

    data = MyCSVData(dataname='data/AAPL_Testing.csv')

    cerebro.adddata(data)
    cerebro.addstrategy(SimpleMACDStrategy)

    cerebro.broker.setcash(10000.0)
    cerebro.broker.setcommission(commission=0.0)

    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')

    results = cerebro.run()
    strat = results[0]

    # Results
    print('\n--- Backtest Results ---')
    print(f"Final Portfolio Value: ${cerebro.broker.getvalue():.2f}")
    print(f"Total Return: {(cerebro.broker.getvalue()/10000 - 1)*100:.2f}%")

    sharpe = strat.analyzers.sharpe.get_analysis().get('sharperatio')
    if sharpe is not None:
        print(f"Sharpe Ratio: {sharpe:.2f}")
    else:
        print("Sharpe Ratio: N/A")

    print(f"Max Drawdown: {strat.analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")
    print(f"Total Return (From Analyzer): {strat.analyzers.returns.get_analysis()['rnorm100']:.2f}%")

    # Plot
    cerebro.plot(style='candlestick')  # Optional: 'line' or 'candlestick'


In [13]:
run_backtest()

Available cash: 10000.0, Calculated size: 404
Attempting to BUY
BUY executed, Price: 24.63, Cost: 9950.52
Attempting to SELL
SELL executed, Price: 23.33, Cost: 9950.52
Available cash: 9474.8, Calculated size: 379
Attempting to BUY
BUY executed, Price: 24.93, Cost: 9448.47
Attempting to SELL
SELL executed, Price: 24.44, Cost: 9448.47
Available cash: 9289.09, Calculated size: 360
Attempting to BUY
BUY executed, Price: 24.95, Cost: 8982.00
Attempting to SELL
SELL executed, Price: 25.95, Cost: 8982.00
Available cash: 9649.09, Calculated size: 341
Attempting to BUY
Order Canceled/Margin/Rejected
Available cash: 9649.09, Calculated size: 341
Attempting to BUY
Order Canceled/Margin/Rejected
Available cash: 9649.09, Calculated size: 344
Attempting to BUY
Order Canceled/Margin/Rejected
Available cash: 9649.09, Calculated size: 333
Attempting to BUY
BUY executed, Price: 28.50, Cost: 9490.50
Attempting to SELL
SELL executed, Price: 29.07, Cost: 9490.50
Available cash: 9838.9, Calculated size: 296

<IPython.core.display.Javascript object>

**Disclaimer: This was not an actual profitable trading strategy, just a test for backtrader**